# **Adding libraries**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import matplotlib.pyplot as plt
from skimage.transform import resize
from matplotlib.pyplot import imread,imshow
import warnings

warnings.filterwarnings('ignore')

# **Data Preprocessing**

In [ ]:
train_cat_list = glob.glob("training_set/training_set/cats/*.jpg")
train_dog_list = glob.glob("training_set/training_set/dogs/*.jpg")

test_cat_list = glob.glob("../input/cat-and-dog/test_set/test_set/cats/*.jpg")
test_dog_list = glob.glob("../input/cat-and-dog/test_set/test_set/dogs/*.jpg")

y_train = np.concatenate((np.zeros((1, len(train_cat_list))), np.ones((1,len(train_dog_list)), dtype=int)), axis=1)
y_test = np.concatenate((np.zeros((1, len(test_cat_list))), np.ones((1,len(test_dog_list)), dtype=int)), axis=1)

X = train_cat_list + train_dog_list + test_cat_list + test_dog_list
Y = np.concatenate((y_train, y_test),axis=1).T

x =[]

for image in X:
    x.append(resize(imread(image),(64,64,1)))


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(np.array(x), Y, test_size = 0.23, random_state = 53)

X_train = X_train.reshape((X_train.shape[0],64*64)).T
X_test = X_test.reshape((X_test.shape[0],64*64)).T

y_train = y_train.T
y_test = y_test.T

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# **Logistic Regression without Sklearn**

**Initializing Parameters**

In [ ]:
def initialize_weight_and_bias(dimension):
    w = np.full((dimension,1),0.01)
    b = 0.0
    return w,b

**Sigmoid Function**

In [ ]:
def sigmoid(z):
    y_head = 1/(1+np.exp(-z))
    return y_head

**Forward Propagation**

In [ ]:
def forward_propagation(w, b, X_train, y_train):
    z = np.dot(w.T, X_train) + b
    y_head = sigmoid(z)
    loss =(-(1 -  y_train) * np.log(1 - y_head)) - (y_train * np.log(y_head))
    cost = (np.sum(loss))/X_train.shape[1]
    return cost,y_head

**Backward Propagation**

In [ ]:
def backward_propagation(X_train, y_head, y_train ):
    derivative_weight = (np.dot(X_train,((y_head-y_train).T)))/X_train.shape[1] # x_train.shape[1]  is for scaling
    derivative_bias = np.sum(y_head-y_train)/X_train.shape[1]                 # x_train.shape[1]  is for scaling
    gradients = {"derivative_weight": derivative_weight,"derivative_bias": derivative_bias}
    return gradients

**Updating Parameters**

In [ ]:
def update(w, b, X_train, y_train, learning_rate, number_of_iteration):
    cost_list = []
    cost_list2 = []
    index = []
    
    for i in range(number_of_iteration):
        cost, y_head = forward_propagation(w, b, X_train, y_train)
        gradients = backward_propagation(X_train, y_head, y_train)
        cost_list.append(cost)
        w = w - learning_rate * gradients["derivative_weight"]
        b = b - learning_rate * gradients["derivative_bias"]
        
        if i % 20 == 0:
            cost_list2.append(cost)
            index.append(i)
            # print("Cost after iteration %i : %f" %(i, cost))
    
    parameters = {"Weight": w, "Bias": b}
    plt.plot(index, cost_list2)
    plt.xticks(index, rotation='vertical')
    plt.xlabel("Number of iteration")
    plt.ylabel("Cost")
    plt.show()
    
    return parameters, gradients, cost_list

**Prediction**

In [ ]:
def predict(w, b, x_test):
    y_head = sigmoid(np.dot(w.T, x_test) + b)
    Y_prediction = np.zeros((1,x_test.shape[1]))
    
    for i in range(y_head.shape[1]):
        if y_head[0, i] <= 0.5:
            Y_prediction[0,i] = 0
        else:
            Y_prediction[0,i] = 1
    
    return Y_prediction
# predict(parameters["weight"],parameters["bias"],x_test)

**Logistic Regression Function**

In [ ]:
def logistic_regression(X_train, y_train, X_test, y_test, learning_rate, number_of_iterations):
    dimension = X_train.shape[0]
    w,b = initialize_weight_and_bias(dimension)
    
    parameters, gradients, cost_list = update(w, b, X_train, y_train, learning_rate, number_of_iterations)
    
    y_prediction_test = predict (parameters["Weight"], parameters["Bias"], X_test)
    y_prediction_train = predict (parameters["Weight"], parameters["Bias"], X_train)
    
    print("train accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_train - y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_test - y_test)) * 100))

logistic_regression(X_train, y_train, X_test, y_test, learning_rate = 0.001, number_of_iterations = 150)

# **Logistic Regression with Sklearn**

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state = 0, max_iter = 150, solver = 'liblinear')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print("Test Accuracy: ", (clf.fit(X_train.T, y_train.T).score(X_test.T, y_test.T)))